In [181]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import tensorflow as tf
from collections import defaultdict
from InceptionTF import *

In [6]:
#Create folder for storing the frames
 
splitfiledir = r"..\ucfTrainTestlist"
splitfile = "mytest.txt"
splitname = splitfile.split('.')[0]

In [7]:
def storeFramesAndFlows(framesdir,splitfiledir,splitfile,splitname):
    
    #Read splitfile
    lines = open(os.path.join(splitfiledir,splitfile),"r")
    for line in lines:
        arr = line.split(" ")
        vidclass = arr[1] 
        line = arr[0].split("/")
        print (line)
        action = line[0]
        filename = line[1]
        actionpath = os.path.join(framesdir,splitname,action)
        framepath = os.path.join(actionpath,filename,"frames")
        flowpath = os.path.join(actionpath,filename,"flows")
        
        #Create folder for Action
        if not os.path.exists(actionpath):
            os.mkdir(actionpath)
        
        #Create folder for videofile
        if not os.path.exists(os.path.join(actionpath,filename)):
            os.mkdir(os.path.join(actionpath,filename))
        
        #Create folder for frames
        if not os.path.exists(framepath):
            os.mkdir(framepath)
            
        #Create folder for flows
        if not os.path.exists(flowpath):
            os.mkdir(flowpath)

        #Read video and collect frames, flows
        vidcap = cv2.VideoCapture(os.path.join(path,action,filename))
        count = 0 
        prevFrame = None
        nextFrame = None
        while True:
            success,image = vidcap.read()
            if not success:
                break
            frame = "frame_%d.jpg"%count
            flow_x = "flow_x_%d.jpg"%count
            flow_y = "flow_y_%d.jpg"%count
            framename = os.path.join(framepath,frame)
            flowname_x = os.path.join(flowpath,flow_x)
            flowname_y = os.path.join(flowpath,flow_y)
            if count == 0:
                prevFrame = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                cv2.imwrite(framename,image)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                count += 1
                continue
            
            nextFrame = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
            cv2.imwrite(framename,image)
            optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()
            flow = optical_flow.calc(prevFrame, nextFrame, None)
            prevFrame = nextFrame
            flow[...,0] = cv2.normalize(flow[...,0],None,0,255,cv2.NORM_MINMAX)
            flow[...,1] = cv2.normalize(flow[...,1],None,0,255,cv2.NORM_MINMAX)
            cv2.imwrite(flowname_x,flow[...,0])
            cv2.imwrite(flowname_y,flow[...,1])

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            count += 1
            
        filename = os.path.join(actionpath,filename,"info.txt")
        #Store the frames count in txt file
        rate = open(filename,"w")
        rate.write("frames:"+str(count))
        rate.write("\n")
        rate.write("class:"+vidclass)
        rate.close()

        #Close the video object
        vidcap.release()

    print ("Stored")

In [8]:
#Store Frames
framesdir = r"..\FramesFlows"

path = r"E:\capstone_adbi_data\UCF-101"


if not os.path.exists(os.path.join(framesdir,splitname)):
    os.mkdir(os.path.join(framesdir,splitname))
    storeFramesAndFlows(framesdir,splitfiledir,splitfile,splitname)
else:
    print ("Frames stored already!")

Frames stored already!


In [134]:
def getFF(classpath):
    frames = list()
    xflows = list()
    yflows = list()
    #Stack up frames and flows
    min_frames = float('inf')
    min_flows = float('inf')
    y_true = None
    for folder in os.listdir(classpath):
        folderpath = os.path.join(classpath,folder)
        # infopath = folderpath\info.txt
        infopath = os.path.join(folderpath,"info.txt")
        # Read info.txt for extracting class
        f = open(infopath,"r")
        y_true = int(f.readlines()[1].strip().split(':')[1])

        # Collect frames
        imgpath = os.path.join(folderpath,"frames")
        flowspath = os.path.join(folderpath,"flows")

        allframes = os.listdir(imgpath)
        allflows = os.listdir(flowspath)
        min_frames = min(len(allframes),min_frames)

        #Sort frames sequentially
        allframes.sort(key = lambda x: int(x.split('_')[1].split('.')[0]))

        #Sort flows sequentially
        allxflows = list(filter(lambda k: k.split('_')[1] == 'x',allflows))
        allyflows = list(filter(lambda k: k.split('_')[1] == 'y',allflows))
        allxflows.sort(key = lambda x: int(x.split('_')[2].split('.')[0]))
        allyflows.sort(key = lambda x: int(x.split('_')[2].split('.')[0]))
        min_flows = min(len(allxflows),min_flows)

        stack = list()
        for frame in allframes:
            img = cv2.imread(os.path.join(imgpath,frame))
            grayimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            stack.append(grayimg)
        frames.append(np.array(stack))
        
        #Collect flows
        xstack = list()
        ystack = list()
        for xflow,yflow in zip(allxflows,allyflows):
            xfl = cv2.imread(os.path.join(flowspath,xflow))
            yfl = cv2.imread(os.path.join(flowspath,yflow))
            grayxfl = cv2.cvtColor(xfl,cv2.COLOR_BGR2GRAY)
            grayyfl = cv2.cvtColor(yfl,cv2.COLOR_BGR2GRAY)
            xstack.append(grayxfl)
            ystack.append(grayyfl)
        xflows.append(np.array(xstack))
        yflows.append(np.array(ystack))
        
    return frames, xflows, yflows, y_true


In [135]:
dfpath = r"../FramesFlows/mytest"
frames_dict = dict()
xflows_dict = dict()
yflows_dict = dict()
for each in os.listdir(dfpath):
    classpath = os.path.join(dfpath,each)
    frames, xflows, yflows, class_name = getFF(classpath)
    frames_dict[class_name] = frames
    xflows_dict[class_name] = xflows
    yflows_dict[class_name] = yflows

In [139]:
#Dictionary key is the class
#Second index represents video number in that class (starting from 0)
#Shape shows NO_OF_FRAMES*HEIGHT_OF_IMAGE*WIDTH_OF_IMAGE
frames_dict[2][0].shape

(149, 240, 320)

In [144]:
#Decided to keep number of segments as 3 for now (consistent with the paper)
segments = 3

In [187]:
def one_hot_encode(data, classes = 101):
    """
    :param data: data to be one hot encoded
    :return: np array with one hot encoding
    """
#     labels = np.zeros(classes)
#     labels[value-1] = 1
#     return labels
    labels = np.zeros((data.size, classes))
    labels[np.arange(data.size), data - 1] = 1
    return labels

def getSegments(frames_dict,segments=3):
    #Get k(equal to segments) random samples(snippets) from frames
    framesSegments = defaultdict(list)
    ySegments = defaultdict(list)
    np.random.seed(0)
    for class_name in frames_dict.keys():
        videos = frames_dict[class_name]
        for video in videos:
            vsegs = np.array_split(video,segments)
            for x in range(len(vsegs)):
                idx = np.random.randint(vsegs[x].shape[0])
                #Append random snippet to segments dictionary
                framesSegments[x].append(vsegs[x][idx])
                ySegments[x].append(class_name)
    
    #One hot encoding
    for x in range(segments):
        ySegments[x] = one_hot_encode(np.array(ySegments[x]))

    return framesSegments, ySegments

In [196]:
framesSegments, yFramesTrueSegments = getSegments(frames_dict)
xFlowsSegments, yFlowsTrueSegments = getSegments(xflows_dict)
yFlowsSegments, yFlowsTrueSegments = getSegments(yflows_dict)

In [198]:
#key is segment number(0,1...segments) and second index in the image
framesSegments[0][1].shape
#key is segment number(0,1,...segments) and second index returns one hot encoded class for the respective image
yFramesTrueSegments[2][4]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [222]:
#We need separate networks (equal to segments) working in parallel 
#Parameters for BN-Inception network
#We need Inputs: a tensor of shape [batch_size, height, width, channels].
imageHeight = 240
imageWidth = 320
colorChannels = 1
num_classes = 101
learning_rate = 0.001
X = tf.placeholder(tf.float64,[None,imageHeight,imageWidth,colorChannels])
y = tf.placeholder(tf.float64,[None,num_classes])

In [223]:
def getLogits(learning_rate,imageHeight, imageWidth, colorChannels, num_classes):
    logits, end_points = inception_v2(inputs = X,num_classes = num_classes, reuse=True)
    
    return logits

In [224]:
tf.reset_default_graph()
getLogits = getLogits(learning_rate,imageHeight, imageWidth, colorChannels, num_classes)

ValueError: Variable InceptionV2/Conv2d_1a_7x7/depthwise_weights does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?